In [1]:
import pymongo
import time
import pandas as pd
import json
import nltk

# Conexão com o MongoDB

In [2]:
client = pymongo.MongoClient()

# database
db = client['8M2020_samples']  

# collection
collection = db.tweets_pt

In [3]:
pipeline = [
    {
        '$project': {
            'tweet_id': '$id_str', 
            'date': {
                '$dateFromString': {
                    'dateString': '$created_at'
                }
            }, 
            'tweet_text': {
                '$ifNull': [
                    '$retweeted_status.full_text', '$full_text'
                ]
            }, 
            'num_likes': '$favorite_count', 
            'num_retweets': '$retweet_count', 
            'quoted_status_id': {
                '$ifNull': [
                    '$quoted_status_id_str', None
                ]
            }, 
            'retweeted_status_id': {
                '$ifNull': [
                    '$retweeted_status.id_str', None
                ]
            }, 
            'reply_to_user': {
                '$ifNull': [
                    '$in_reply_to_user_id_str', None
                ]
            }, 
            'reply_to_status': {
                '$ifNull': [
                    '$in_reply_to_status_id_str', None
                ]
            }, 
            'user_id': '$user.id_str', 
            'screen_name': '$user.screen_name', 
            'followers': '$user.followers_count', 
            'following': '$user.friends_count'
        }
    }, {
        '$project': {
            '_id': 0, 
            'tweet_id': 1, 
            'date': {
                '$dateToString': {
                    'format': '%d/%m/%Y %H:%M:%S', 
                    'date': '$date'
                }
            }, 
            'tweet_text': 1, 
            'num_likes': 1, 
            'num_retweets': 1, 
            'quoted_status_id': 1, 
            'retweeted_status_id': 1, 
            'reply_to_user': 1, 
            'reply_to_status': 1, 
            'user_id': 1, 
            'screen_name': 1, 
            'followers': 1, 
            'following': 1
        }
    }
]

In [5]:
time_start = time.time()
document = list(collection.aggregate(pipeline=pipeline))
time_end = time.time()

print(
    f'Tempo de execução: {time.strftime("%H:%M:%S", time.gmtime(time_end - time_start))}')
print(f'{len(document)} documentos selecionados')

Tempo de execução: 00:00:07
274636 documentos selecionados


In [6]:
print(f'Amostra de um documento:\n')
print(json.dumps(document[0], sort_keys=False, indent=3, ensure_ascii=False))

Amostra de um documento:

{
   "tweet_id": "1236411302895124487",
   "tweet_text": "Sol com cara O projeto Inclusive, Praia! está de volta no próximo domingo (8) com uma programação especial em referência ao Dia Internacional da Mulher. Vai ter jogos, brincadeiras, diversão e show com Isabelle Passinho. 🏝 Não perca! &gt; https://t.co/fYin5VeZwx \n#GovernoDeTodosNós https://t.co/YIGahguJN0",
   "num_likes": 8,
   "num_retweets": 2,
   "quoted_status_id": null,
   "retweeted_status_id": null,
   "reply_to_user": null,
   "reply_to_status": null,
   "user_id": "53085255",
   "screen_name": "GovernoMA",
   "followers": 175381,
   "following": 166,
   "date": "07/03/2020 22:00:00"
}


In [7]:
df = pd.DataFrame(document)
df = df[['tweet_id', 'date', 'tweet_text', 'num_likes', 'num_retweets',
                   'retweeted_status_id', 'quoted_status_id', 'reply_to_user', 'reply_to_status',
                   'user_id', 'screen_name', 'followers', 'following']]
df.head()

,tweet_id,date,tweet_text,num_likes,num_retweets,retweeted_status_id,quoted_status_id,reply_to_user,reply_to_status,user_id,screen_name,followers,following
0,1236411302895124487,07/03/2020 22:00:00,"Sol com cara O projeto Inclusive, Praia! está ...",8,2,None,None,None,None,53085255,GovernoMA,175381,166
1,1236411378879037442,07/03/2020 22:00:18,Hoje é dia de transmissão especial na Fla TV!!...,0,36,1236371617732874242,None,None,None,79617245,audreysteen,38,342
2,1236411364643659776,07/03/2020 22:00:15,"A presidenta nacional do PT, deputada @gleisi ...",0,184,1236375681245876225,None,None,None,139207105,ProfValterCosta,1758,2800
3,1236411353444868096,07/03/2020 22:00:12,Cartaz de minha enteada querida(linda e talent...,0,3,1236407917554929666,None,None,None,1093274352060121088,purpurediamond,28,90
4,1236411366086508544,07/03/2020 22:00:15,Dia Internacional da Mulher: o que os homens p...,2,0,None,None,None,None,126318634,professoraAlice,26485,170


In [8]:
print(f'Amostra de um tweet antes do pré-processamento:\n\n{df.tweet_text[0]}')

Amostra de um tweet antes do pré-processamento:

Sol com cara O projeto Inclusive, Praia! está de volta no próximo domingo (8) com uma programação especial em referência ao Dia Internacional da Mulher. Vai ter jogos, brincadeiras, diversão e show com Isabelle Passinho. 🏝 Não perca! &gt; https://t.co/fYin5VeZwx 
#GovernoDeTodosNós https://t.co/YIGahguJN0


In [9]:
df.to_csv('tweets_pt.csv')

## Pré-processamento

In [10]:
# remover links
df['clean_text'] = df['tweet_text'].str.replace(r"http\S+","") 

In [11]:
# remover mentions
df['clean_text'] = df['clean_text'].str.replace(r"@\S+","")

In [12]:
# remover hashtags
df['clean_text'] = df['clean_text'].str.replace(r"#\S+"," ")

In [13]:
# remover quebra de linhas
df['clean_text'] = df['clean_text'].str.replace(r"\n"," ")

In [14]:
# remover pontuações
df['clean_text'] = df['clean_text'].str.replace(r"[^\w\s]|_"," ")

In [15]:
# remover números
df['clean_text'] = df['clean_text'].str.strip().str.replace(r"[0-9]+(\s|\w)","")

In [16]:
# remover espaços duplos
df['clean_text'] = df['clean_text'].str.strip().str.replace(r"\s{2,}"," ")

In [17]:
# converter todas as letras para minusculas
df['clean_text'] = df['clean_text'].str.lower()

In [18]:
# remover letras que se repetem em sequência mais de 3x
import string
alphabet = list(string.ascii_lowercase)

for letter in alphabet:
    pattern = '{}{}'.format(letter, '{3,}')
    df['clean_text'] = df['clean_text'].str.replace(r'{}'.format(pattern), letter)

In [19]:
# remover stpwords
with open('portuguese.txt', 'r') as file:
    stopwords = file.readlines()
stopwords = [sw.replace('\n','') for sw in stopwords]

def remover_stopwords(texto):
    palavras = [i for i in texto.split() if not i in stopwords]
    return (" ".join(palavras))

df['clean_text'] = df['clean_text'].apply(remover_stopwords)

In [20]:
# remover palavras com 2 caracteres ou menos
def remover_len_2(texto):
    palavras = [i for i in texto.split() if len(i) > 2]
    return (" ".join(palavras))

df['clean_text'] = df['clean_text'].apply(remover_len_2)

In [21]:
print(
    f'Amostra de um tweet depois do pré-processamento:\n\n{df.tweet_text[0]}\n\n{df.clean_text[0]}')

Amostra de um tweet depois do pré-processamento:

Sol com cara O projeto Inclusive, Praia! está de volta no próximo domingo (8) com uma programação especial em referência ao Dia Internacional da Mulher. Vai ter jogos, brincadeiras, diversão e show com Isabelle Passinho. 🏝 Não perca! &gt; https://t.co/fYin5VeZwx 
#GovernoDeTodosNós https://t.co/YIGahguJN0

sol cara projeto inclusive praia volta próximo domingo programação especial referência dia internacional mulher vai ter jogos brincadeiras diversão show isabelle passinho perca


In [22]:
def reduzir_radical(texto):
    stemmer = nltk.stem.RSLPStemmer()
    palavras = [stemmer.stem(i) for i in texto.split()]
    return (" ".join(palavras))

In [23]:
reduzir_radical(df.clean_text[0])

'sol car projet inclusiv prai volt próx doming program espec refer dia internac mulh vai ter jog brinc divers show isabell pass perc'

In [24]:
# df['tweet_text'] = df['tweet_text'].apply(reduzir_radical)

In [25]:
import unicodedata

def remover_acentuacao(text):
    # Unicode normalize transforma um caracter em seu equivalente em latin.
    nfkd = unicodedata.normalize('NFKD', text)
    palavra_sem_acento = u"".join([c for c in nfkd if not unicodedata.combining(c)])
    return (palavra_sem_acento)

In [26]:
remover_acentuacao(df.clean_text[0])

'sol cara projeto inclusive praia volta proximo domingo programacao especial referencia dia internacional mulher vai ter jogos brincadeiras diversao show isabelle passinho perca'

In [27]:
df['clean_text'] = df['clean_text'].apply(remover_acentuacao)

In [28]:
print(
    f'Amostra de um tweet depois do pré-processamento:\n\n{df.tweet_text[0]}\n\n{df.clean_text[0]}')

Amostra de um tweet depois do pré-processamento:

Sol com cara O projeto Inclusive, Praia! está de volta no próximo domingo (8) com uma programação especial em referência ao Dia Internacional da Mulher. Vai ter jogos, brincadeiras, diversão e show com Isabelle Passinho. 🏝 Não perca! &gt; https://t.co/fYin5VeZwx 
#GovernoDeTodosNós https://t.co/YIGahguJN0

sol cara projeto inclusive praia volta proximo domingo programacao especial referencia dia internacional mulher vai ter jogos brincadeiras diversao show isabelle passinho perca


### Etapa opcional

O atributo 'lang' presentes nos documentos podem ter sido atribuidos erroneamente. Objetivando eliminar tweets que possam ser de um idioma não requisitado, desenvolveram-se 2 funções para ajudar a confirmar o idioma destes.

Para desabilitar warnings:

In [ ]:
import logging
logging.disable(logging.WARNING)

#### 1. Usando polyglot

In [ ]:
import polyglot
from polyglot.detect import Detector

In [ ]:
def get_polyglot_language(text):
    try:
        detector = Detector(text)
    except polyglot.detect.base.UnknownLanguage:
        lang = 'Undefined'
    else:
        lang = detector.language.name
    
    return (lang)

In [ ]:
get_polyglot_language(df['tweet_text'][0])

In [ ]:
df['polyglot_lang'] = df['tweet_text'].apply(get_polyglot_language)

In [ ]:
df['polyglot_lang'].value_counts()

#### 2. Usando textblob

In [ ]:
import textblob
from textblob import TextBlob
import pycountry

In [ ]:
def get_textblob_language(text):
    try:
        time.sleep(0.2)
        b = TextBlob(text)
        language = pycountry.languages.get(alpha_2=b.detect_language())
        language_name = language.name
    except textblob.exceptions.TranslatorError:
        # language_name = "Deu ruim"
        language_name = 'Undefined'
    
    return(language_name) 

In [ ]:
get_textblob_language(df['tweet_text'][0])

In [ ]:
df['textblob_lang'] = df['tweet_text'].apply(get_textblob_language)

Para reabilitar warnings:

In [ ]:
logging.disable(logging.NOTSET)

In [ ]:
df.shape

# Criação do arquivo .csv

### Cenário 1: base completa

Caso tenha se utilizado umas das duas funções - **get_polyglot_language**, por exemplo - use o seguinte trecho para gerar o arquivo .csv:

In [ ]:
index = df['polyglot_lang'] == 'Portuguese'
new_df = df[index]
new_df = new_df.drop('polyglot_lang', axis=1)

In [ ]:
new_df.to_csv('cenario1_8M2020_polyglot_tweets_pt.csv', index=False)

Caso contrário:

In [ ]:
df = df.drop('polyglot_lang', axis=1)

In [29]:
df = df[df['clean_text'] != '']

In [30]:
df.shape

(274297, 14)

In [31]:
df.to_csv('cenario1_8M2020_tweets_pt.csv', index=False)

criação do arquivo .txt (somente com os textos dos tweets):

In [32]:
import numpy as np

In [33]:
tweets_text = np.array(df['clean_text'])

In [34]:
np.savetxt(fname='cenario1_8M2020_tweets_pt.txt', X=tweets_text, fmt='%s')

## Cenário 2: tweets sem duplicatas

In [35]:
df.drop_duplicates(['clean_text'], inplace = True)

In [36]:
df.shape

(40324, 14)

In [37]:
df.to_csv('cenario2_8M2020_tweets_pt.csv', index=False)

In [38]:
tweets_text = np.array(df['clean_text'])

In [39]:
np.savetxt(fname='cenario2_8M2020_tweets_pt.txt', X=tweets_text, fmt='%s')